In [201]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier


In [202]:
df_train = pd.read_csv('downloaded/train.csv')

y_train = df_train['Survived']

# skipping non numeric columns and columns that have nulls in some rows
# TODO need better solution for nulls and numeric columns
# Age has some null columns
# Fare has 1 null in testing data
# eventually should probably be using of these columns 
# 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked'
training_columns = ['Pclass', 'SibSp', 'Parch']

x_train = df_train[training_columns]

df_test = pd.read_csv('downloaded/test.csv')
x_test = df_test[training_columns]

x_train.describe()


,Pclass,SibSp,Parch
count,891.000000,891.000000,891.000000
mean,2.308642,0.523008,0.381594
std,0.836071,1.102743,0.806057
min,1.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000
50%,3.000000,0.000000,0.000000
75%,3.000000,1.000000,0.000000
max,3.000000,8.000000,6.000000


In [203]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
tree.fit(x_train.as_matrix(), y_train.as_matrix())
x_test.describe()

,Pclass,SibSp,Parch
count,418.000000,418.000000,418.000000
mean,2.265550,0.447368,0.392344
std,0.841838,0.896760,0.981429
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,3.000000,0.000000,0.000000
75%,3.000000,1.000000,0.000000
max,3.000000,8.000000,9.000000


In [204]:
result = tree.predict(x_test)
result

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0,

In [205]:
# last step is output the results to a csv
df_for_file = df_test[['PassengerId',]]
df_result = pd.DataFrame(result)
df_result.columns = ['Survived']

# get id of false pandas warning
pd.options.mode.chained_assignment = None

df_for_file['Survived'] = df_result[['Survived',]]

df_for_file.to_csv("out.csv", header = ['PassengerId', 'Survived'], index=False)

In [206]:
df_for_file

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,1
8,900,0
9,901,0
